In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

print('Readyyy')

Readyyy


In [2]:
train_df = pd.read_excel("Data_Train.xlsx")
test_df = pd.read_excel("Test_set.xlsx")

In [3]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


**Importance of various features**

* **Date_Of_Journey** : 
    * The price of flight tickets depend on the month of travel.
    * The price of tickets are higher on Mondays, Wednesdays and Thursdays on buisness routes.
        * To get these infos I will convert dtype to dt object.
* **Route and Total_Stops** :
    * Len(Route) for a particular flight is same as the total_stops for that flight and hence a redundant feature.
    * I'll drop Route feature and convert Total_Stops feature to numerical.
* **Dep_Time and Duration** :
    * Will convert Dep_Time to Dep_time_hrs and Dep_Time_min.
    * Duration is in %h %m format, will convert it to %m format.
        * Now that we have departure time and duration of flight, Arrival_Time feature has become redundant and hence needs to be dropped.

In [4]:
# Check for NULL values.

train_df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [5]:
test_df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [6]:
train_df[train_df['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


There is only one flight with **NULL** value in the train set, so will remove that row and then merge the two df into big_df.

In [7]:
train_df.drop([9039], axis = 0, inplace = True)

In [8]:
# Merge train_df, test_df into one df for data cleaning and feature engineering.

big_df = pd.concat([train_df, test_df], axis = 0, sort = False)

big_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


### Feature Engineering n Data Cleaning

**Date_Of_Journey**

In [9]:
doj = pd.to_datetime(big_df['Date_of_Journey'], format = '%d/%m/%Y')

In [10]:
big_df['Date'] = doj.dt.day

In [11]:
big_df['Month'] = doj.dt.month

In [12]:
big_df['Weekday'] = doj.dt.weekday

Monday -> 0 ..... Sunday -> 6

In [13]:
big_df.drop(columns = ['Date_of_Journey','Route', 'Arrival_Time'], inplace = True)

In [14]:
big_df['Weekday'].replace([0, 1, 2, 3, 4, 5, 6], ['M', 'Tu', 'W', 'Th', 'F', 'Sa', 'Su'], inplace  = True)

In [15]:
weekdays = pd.get_dummies(big_df['Weekday'], drop_first = True)
big_df = pd.concat([big_df, weekdays], axis = 1)
big_df.drop(columns = ['Weekday'], inplace = True)

In [16]:
big_df.head()

,Airline,Source,Destination,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,M,Sa,Su,Th,Tu,W
0,IndiGo,Banglore,New Delhi,22:20,2h 50m,non-stop,No info,3897.0,24,3,0,0,1,0,0,0
1,Air India,Kolkata,Banglore,05:50,7h 25m,2 stops,No info,7662.0,1,5,0,0,0,0,0,1
2,Jet Airways,Delhi,Cochin,09:25,19h,2 stops,No info,13882.0,9,6,0,0,1,0,0,0
3,IndiGo,Kolkata,Banglore,18:05,5h 25m,1 stop,No info,6218.0,12,5,0,0,1,0,0,0
4,IndiGo,Banglore,New Delhi,16:50,4h 45m,1 stop,No info,13302.0,1,3,0,0,0,0,0,0


**Dep_Time**

In [17]:
big_dep = big_df['Dep_Time']

big_dep_hrs = []
big_dep_min = []


for d in big_dep:
    h,m = d.split(':')[0], d.split(':')[1]
    h = int(h)
    m = int(m)
    big_dep_hrs.append(h)
    big_dep_min.append(m)
    
big_df['Dep_hrs'] = big_dep_hrs
big_df['Dep_min'] = big_dep_min

big_df.drop(columns = ['Dep_Time'], inplace = True)

**Total_Stops**

In [18]:
big_df['Total_Stops'].value_counts()

1 stop      7056
non-stop    4340
2 stops     1899
3 stops       56
4 stops        2
Name: Total_Stops, dtype: int64

In [19]:
big_df['Total_Stops'].replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace = True)

**Additional_Info**

In [20]:
big_df['Additional_Info'].value_counts()

No info                         10492
In-flight meal not included      2426
No check-in baggage included      396
1 Long layover                     20
Change airports                     8
Business class                      5
No Info                             3
1 Short layover                     1
Red-eye flight                      1
2 Long layover                      1
Name: Additional_Info, dtype: int64

In [21]:
big_df['Additional_Info'].replace('No Info', 'No info', inplace = True)
big_df['Additional_Info'].replace(['No check-in baggage included', '1 Long layover', 'Change airports'
                                  , 'Business class', '2 Long layover', '1 Short layover', 'Red-eye flight'],
                                  'other', inplace = True)

In [22]:
info = pd.get_dummies(big_df['Additional_Info'], drop_first = True)
big_df = pd.concat([big_df, info], axis = 1)
big_df.drop(columns = ['Additional_Info'], inplace = True)

**Duration**

In [23]:
big_duration = big_df['Duration']

big_duration_min = []

for d in big_duration:
    hrs = d.split()[0]
    
    minutes = int(hrs[:-1]) * 60
    m = 0
    
    if len(d.split()) == 2:
        m = d.split()[1]
        m = int(m[:-1])
    
    if m:
        big_duration_min.append(minutes + m)
    else:
        big_duration_min.append(minutes)
        
big_df['Duration_min'] = big_duration_min
big_df.drop(columns = ['Duration'], inplace = True)

In [32]:
big_df.head()

,Total_Stops,Price,Date,Month,M,Sa,Su,Th,Tu,W,...,dest__Delhi,dest__Hyderabad,dest__Kolkata,Air India,IndiGo,Jet Airways,Multiple carriers,SpiceJet,Vistara,other_flight
0,0,3897.0,24,3,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,2,7662.0,1,5,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,2,13882.0,9,6,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,6218.0,12,5,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,13302.0,1,3,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


**Destionation and Source**

In [25]:
big_df['Source'].value_counts()

Delhi       5681
Kolkata     3581
Banglore    2752
Mumbai       883
Chennai      456
Name: Source, dtype: int64

In [26]:
big_df['Destination'].value_counts()

Cochin       5681
Banglore     3581
Delhi        1582
New Delhi    1170
Hyderabad     883
Kolkata       456
Name: Destination, dtype: int64

In [27]:
big_df['Destination'].replace('New Delhi', 'Delhi', inplace = True)

In [28]:
src = pd.get_dummies(big_df['Source'], drop_first = True, prefix = 'src_')
dest = pd.get_dummies(big_df['Destination'], drop_first = True, prefix = 'dest_')

big_df = pd.concat([big_df, src, dest], axis = 1)

In [29]:
big_df.drop(columns = ['Source', 'Destination'], inplace = True)

**Airline**

In [30]:
big_df.Airline.value_counts()

Jet Airways                          4746
IndiGo                               2564
Air India                            2191
Multiple carriers                    1543
SpiceJet                             1026
Vistara                               608
Air Asia                              405
GoAir                                 240
Multiple carriers Premium economy      16
Jet Airways Business                    8
Vistara Premium economy                 5
Trujet                                  1
Name: Airline, dtype: int64

In [31]:
big_df.Airline.replace(['GoAir', 'Multiple carriers Premium economy', 'Jet Airways Business', 'Vistara Premium economy', 'Trujet']
                      , 'other_flight', inplace = True)

airline = pd.get_dummies(big_df['Airline'], drop_first = True)

big_df = pd.concat([big_df, airline], axis = 1)
big_df.drop(columns = ['Airline'], inplace = True)

## All set now

In [33]:
big_df.dtypes

Total_Stops            int64
Price                float64
Date                   int64
Month                  int64
M                      uint8
Sa                     uint8
Su                     uint8
Th                     uint8
Tu                     uint8
W                      uint8
Dep_hrs                int64
Dep_min                int64
No info                uint8
other                  uint8
Duration_min           int64
src__Chennai           uint8
src__Delhi             uint8
src__Kolkata           uint8
src__Mumbai            uint8
dest__Cochin           uint8
dest__Delhi            uint8
dest__Hyderabad        uint8
dest__Kolkata          uint8
Air India              uint8
IndiGo                 uint8
Jet Airways            uint8
Multiple carriers      uint8
SpiceJet               uint8
Vistara                uint8
other_flight           uint8
dtype: object

In [34]:
train_df = big_df[: train_df.shape[0]]
test_df = big_df[train_df.shape[0]:]

In [35]:
X = train_df.drop(columns = ['Price'])
y = train_df['Price']

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 7)

In [38]:
from sklearn.ensemble import RandomForestRegressor
rr = RandomForestRegressor(n_estimators = 500,random_state = 7)
rr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=7, verbose=0,
                      warm_start=False)

In [40]:
y_pred = rr.predict(X_val)

def rmsle(Y,YH):
    s = 0
    for y,yh in zip(Y,YH):
        s += (np.log(y)-np.log(yh)) ** 2
    return (s / (Y.shape[0])) ** 0.5

1 - rmsle(y_val,y_pred)

0.867976843198199

In [42]:
X_test = test_df.drop(columns = ['Price'])

In [45]:
y_pred_test = rr.predict(X_test)
y_pred_test = y_pred_test.astype(int)

submission = pd.DataFrame({
    'Price': y_pred_test
})

submission.to_excel("flight_prediction_1.xlsx", index = False)

	
**Result On Submission**        
Avi Kasliwal	0.9388554	0.9388554	19/02/2020

In [48]:
train_df.to_excel('train_df.xlsx', index = False)
test_df.to_excel('test_df.xlsx', index = False)